In [1]:
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore") 
import datetime

In [2]:
today=str(datetime.datetime.today().strftime('%d_%m_%Y'))

In [3]:
incident_sla = pd.read_excel(r'C:\NotBackedUp\SOE\source_files\INC_SLA\incident_sla.xlsx')

In [4]:
domain_map = pd.read_csv(r'C:\NotBackedUp\SOE\source_files\service_mapping\SIAM_ServiceMapping.csv')
model1_maping = pd.read_csv(r'C:\NotBackedUp\SOE\source_files\service_mapping\model1_mapping.csv')
domain_map = domain_map.loc[domain_map.notnull().sum(1).groupby(domain_map.Name).idxmax()]
model1_maping = model1_maping.loc[model1_maping.notnull().sum(1).groupby(model1_maping.Name).idxmax()]

In [5]:
list(incident_sla)

['Number',
 'Category',
 'SLA definition',
 'Stage',
 'Name',
 'Created',
 'Domain',
 'Has breached',
 'SLA due',
 'Severity']

In [6]:
incident_sla = incident_sla.rename(columns={'Name':'business_service_name'})

In [7]:
incident_sla = incident_sla[['Number','business_service_name','Has breached']]

In [8]:
incident_sla = incident_sla.rename(columns={'Has breached':'Hasbreached'})

In [9]:
BI = incident_sla[(incident_sla['Hasbreached']== True)]

In [10]:
BI['business_service_name'] = BI['business_service_name'].str.strip('[PROD]')
BI['business_service_name'] = BI['business_service_name'].str.strip()

In [11]:
BI = BI.groupby(['business_service_name'])['Number'].count().reset_index(name='noInBreached')

In [12]:
BI

,business_service_name,noInBreached
0,ABS Suite,1
1,AD-LDS (Active Directory Lightweight Directory...,2
2,AI Machine Learning,1
3,AI Retail OFI Bank Statements,1
4,AI Trade Bank Guarantee,6
5,AML NetReveal - Global,19
6,AML/CTF Management - Global,2
7,ANZ App - AU,19
8,ANZ App application Build Service,2
9,ANZ Commissions Platform [Ext],11


In [13]:
BI['business_service_name']=BI['business_service_name'].str.upper()
model1_maping['Name']=model1_maping['Name'].str.upper()
domain_map['Name']=domain_map['Name'].str.upper()

In [14]:
Model1 = BI.merge(model1_maping,how='left',left_on=['business_service_name'],right_on=['Name'])

In [15]:
Model1['Name'] = Model1['Name'].fillna(0)

In [16]:
Model1 = Model1[Model1['Name'] != 0]

In [17]:
Model1 = Model1[['Domain','business_service_name','noInBreached']]

In [18]:
Model1

,Domain,business_service_name,noInBreached
298,Australia Retail and Commercial,MAUDE - AU,17


In [19]:
Model1.to_csv(r'C:\NotBackedUp\SOE\Model1\CSV\IncidentSLA_'+today+'.csv',index=None)

In [20]:
SIAM = BI.merge(domain_map,how='left',left_on=['business_service_name'],right_on=['Name'])

In [21]:
SIAM['Name'] = SIAM['Name'].fillna(0)

In [22]:
SIAM = SIAM[SIAM['Name'] != 0]

In [23]:
SIAM = SIAM[['Domain','business_service_name','noInBreached']]

In [24]:
SIAM.to_csv(r'C:\NotBackedUp\SOE\SIAM\CSV\IncidentSLA_'+today+'.csv',index=None)